In [19]:
import time
import numpy as np
import pandas as pd
import sys
import os
from tqdm.auto import tqdm
import brainpy as bp
import brainpy.math as bm
import traceback

bm.set_platform("cpu")

try:
    parent_dir = os.path.dirname(os.path.abspath(__file__))
except:
    parent_dir = os.path.abspath("..")
print("Parent directory:", parent_dir)

sys.path.append(parent_dir)


from models.network import run_simulation

In [5]:
G_GJ_VALUES = np.linspace(0.0, 0.05, 10)
SEEDS = [42, 123, 99, 7, 2024]
DURATION = 10_000.0
DT = 0.025
BASE_NET_PARAMS = {}

In [21]:
results_subdir = os.path.join(parent_dir, "results", f"io_synchrony_{time.strftime('%m-%d_%H:%M:%S')}")
os.makedirs(results_subdir, exist_ok=True)

In [22]:
results_subdir = os.path.join(parent_dir, "results", f"io_synchrony_{time.strftime('%m-%d_%H:%M:%S')}")
os.makedirs(results_subdir, exist_ok=True)

i = 0
for g_gj in tqdm(G_GJ_VALUES, position=0, desc="g_gj"):
    for seed in tqdm(SEEDS, position=1, desc="seed"):
        current_net_params = BASE_NET_PARAMS.copy()
        current_net_params["IO_g_gj"] = g_gj

        try:
            runner = run_simulation(
                duration=DURATION, dt=DT, net_params=current_net_params, seed=seed
            )
            
            data = {}
            for k in runner.mon:
                data[k] = np.array(runner.mon[k])
            run_path = os.path.join(results_subdir, f"g_gj{g_gj}_seed{seed}.npz")
            np.savez(run_path, **data)

        except Exception as e:
            full_error = traceback.format_exc()
            tqdm.write(f"Error during simulation: {e}\n{full_error}")
        break
    i += 1
    if i == 2:
        break

g_gj:   0%|          | 0/10 [00:00<?, ?it/s]

seed:   0%|          | 0/5 [00:00<?, ?it/s]

Running simulation for 10000.0ms with dt=0.025ms...


  0%|          | 0/400000 [00:00<?, ?it/s]

Simulation finished.


seed:   0%|          | 0/5 [00:00<?, ?it/s]

Running simulation for 10000.0ms with dt=0.025ms...


  0%|          | 0/400000 [00:00<?, ?it/s]

Simulation finished.


In [13]:
# Load
data = np.load("../results/io_synchrony_data.npz")
print(list(data.keys()))
print(data["g_gj=0.0_seed=42_pc.V"].shape)

['g_gj=0.0_seed=42_pf.I_OU', 'g_gj=0.0_seed=42_pc.V', 'g_gj=0.0_seed=42_pc.spike', 'g_gj=0.0_seed=42_pc.w', 'g_gj=0.0_seed=42_pc.input', 'g_gj=0.0_seed=42_cn.V', 'g_gj=0.0_seed=42_cn.spike', 'g_gj=0.0_seed=42_cn.I_PC', 'g_gj=0.0_seed=42_io.V_soma', 'g_gj=0.0_seed=42_io.V_axon', 'g_gj=0.0_seed=42_io.V_dend', 'g_gj=0.0_seed=42_io.input', 'g_gj=0.0_seed=42_io.I_OU', 'g_gj=0.0_seed=42_ts']
(400000, 100)
